# Optimizing corner detection

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os

from ipywidgets import interact

In [ ]:
# we need to access the points of the contour in a circular manner
def get_circular(contour, index):
    """Gets the point at the index of the contour, 
    but wraps around if the index is out of bounds

    Args:
        contour (np.array(len, 2)): array of points
        index (int): index of the point to get

    Returns:
        np.array(2): the point
    """
    # make sure index is positive
    index = index % len(contour) + len(contour)
    return contour[index % len(contour)]

def get_slice_circular(contour, start, end):
    """Gets the slice of the contour from start to end, but
    wraps around if the index is out of bounds

    Args:
        contour (np.array(len, 2)): array of points
        start (int): index of the start of the slice, may be negative
        end (int): index of the end of the slice, may be negative

    Returns:
        np.array(end-start, 2): array of points that make up the slice
    """
    
    # make sure start and end are positive
    start = start % len(contour)
    end = end % len(contour)
    if start < 0:
        start += len(contour)
    if end < 0:
        end += len(contour)
    
    # make sure end is larger than start
    if end < start:
        end += len(contour)
    
    if end > len(contour):
        return np.concatenate((contour[start:], contour[:end - len(contour)]))
    
    return contour[start:end]

def get_contour(puzzle_piece_img):
    """gets the contour of the puzzle piece

    Args:
        puzzle_piece_img (Mat): image of the puzzle piece

    Returns:
        np.array(len, 2): list of points that make up the contour
    """
    img_gray = cv2.cvtColor(puzzle_piece_img, cv2.COLOR_BGR2GRAY)
    # Threshold the image
    _, img_thresh = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    return contours[0][:, 0, :]

In [ ]:
# create a list of all the images in the folder

# Gets the puzzle piece image
def get_piece_image(path):
    # images are of the format piece#.png
    img = cv2.imread(path)
    return img

puzzle_piece_dir = 'dataset/pieces-raw'
image_files = os.listdir('puzzle-pieces')
image_files = sorted(image_files, key=lambda x: int(x[5:-4]))

# load all off the images
piece_images = []
for i in range(len(image_files)):
    piece_images.append(get_piece_image(os.path.join))
    
@interact(index=(0, len(piece_images) - 1))
def view_piece(index):
    plt.imshow(piece_images[index])
    plt.show()